In [0]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Activation, LSTM, BatchNormalization, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow_addons.metrics import F1Score

from sklearn.preprocessing import MinMaxScaler

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_df = pd.read_csv('/content/drive/My Drive/Ion_Channel/Kaggle_dataset/train_clean.csv')
test_df = pd.read_csv('/content/drive/My Drive/Ion_Channel/Kaggle_dataset/test_clean.csv')
train_df.head()

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0


## Random Forest Model training

In [0]:
# 5 models created -- 1s, 1f, 3, 5, 10

# upto 1 cahnnel open (low channel open probability)
rf1s = RandomForestClassifier(n_estimators=1000, max_depth=1)

# upto 1 cahnnel open (high channel open probability)
rf1f = RandomForestClassifier(n_estimators=1000, max_depth=1)

# upto 3 cahnnels open
rf3 = RandomForestClassifier(n_estimators=1000, max_leaf_nodes=4)

# upto 5 cahnnels open
rf5 = RandomForestClassifier(n_estimators=1000, max_leaf_nodes=6)

# upto 10 cahnnels open
rf10 = RandomForestClassifier(n_estimators=1000, max_leaf_nodes=11)

In [0]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

resample = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='not majority'), random_state=100)

In [0]:
X, y = resample.fit_sample(np.array(train_df['signal'][0:1000000]).reshape(-1, 1), np.array(train_df['open_channels'][0:1000000]).reshape(-1, 1))
rf1s.fit(X, y)

In [0]:
rf1f_X = np.concatenate([train_df['signal'].values[1000000: 1500000], train_df['signal'].values[3000000: 3500000]])
rf1f_y = np.concatenate([train_df['open_channels'].values[1000000: 1500000], train_df['open_channels'].values[3000000: 3500000]])

X, y = resample.fit_sample(np.array(rf1f_X).reshape(-1, 1), np.array(rf1f_y).reshape(-1, 1))
rf1f.fit(X, y)

In [0]:
rf3_X = np.concatenate([train_df['signal'].values[1500000: 2000000], train_df['signal'].values[3500000: 4000000]])
rf3_y = np.concatenate([train_df['open_channels'].values[1500000: 2000000], train_df['open_channels'].values[3500000: 4000000]])

X, y = resample.fit_sample(np.array(rf3_X).reshape(-1, 1), np.array(rf3_y).reshape(-1, 1))
rf3.fit(X, y)

In [0]:
rf5_X = np.concatenate([train_df['signal'].values[2500000: 3000000], train_df['signal'].values[4000000: 4500000]])
rf5_y = np.concatenate([train_df['open_channels'].values[2500000: 3000000], train_df['open_channels'].values[4000000: 4500000]])

X, y = resample.fit_sample(np.array(rf5_X).reshape(-1, 1), np.array(rf5_y).reshape(-1, 1))
rf5.fit(X, y)

In [0]:
rf10_y = np.concatenate([train_df['open_channels'].values[2000000: 2500000], train_df['open_channels'].values[4500000: 5000000]])
rf10_y = pd.Series(rf10_y)
index =np.array([rf10_y[rf10_y == 0].index[0], rf10_y[rf10_y == 0].index[1]])

rf10_X = np.concatenate([train_df['signal'].values[2000000: 2500000], train_df['signal'].values[4500000: 5000000]])
rf10_y = np.concatenate([train_df['open_channels'].values[2000000: 2500000], train_df['open_channels'].values[4500000: 5000000]])
rf10_X = np.delete(rf10_X, index)
rf10_y = np.delete(rf10_y, index)
resample = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='not majority', n_neighbors=2), random_state=100)
X, y = resample.fit_sample(np.array(rf10_X).reshape(-1, 1), np.array(rf10_y).reshape(-1, 1))
rf10.fit(X, y)

In [0]:
# Predicitons
sub = pd.read_csv('/content/drive/My Drive/Ion_Channel/Kaggle_dataset/sample_submission.csv')

a = 0 # SUBSAMPLE A, Model 1s
sub.iloc[100000*a:100000*(a+1),1] = rf1s.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 1 # SUBSAMPLE B, Model 3
sub.iloc[100000*a:100000*(a+1),1] = rf3.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 2 # SUBSAMPLE C, Model 5
sub.iloc[100000*a:100000*(a+1),1] = rf5.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 3 # SUBSAMPLE D, Model 1s
sub.iloc[100000*a:100000*(a+1),1] = rf1s.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 4 # SUBSAMPLE E, Model 1f
sub.iloc[100000*a:100000*(a+1),1] = rf1f.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 5 # SUBSAMPLE F, Model 10
sub.iloc[100000*a:100000*(a+1),1] = rf10.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 6 # SUBSAMPLE G, Model 5
sub.iloc[100000*a:100000*(a+1),1] = rf5.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 7 # SUBSAMPLE H, Model 10
sub.iloc[100000*a:100000*(a+1),1] = rf10.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 8 # SUBSAMPLE I, Model 1s
sub.iloc[100000*a:100000*(a+1),1] = rf1s.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

a = 9 # SUBSAMPLE J, Model 3
sub.iloc[100000*a:100000*(a+1),1] = rf3.predict(test_df['signal'].values[100000*a:100000*(a+1)].reshape((-1,1)))

 # BATCHES 3 AND 4, Model 1s
sub.iloc[1000000:2000000,1] = rf1s.predict(test_df['signal'].values[1000000:2000000].reshape((-1,1)))

sub.to_csv('/content/drive/My Drive/Ion_Channel/Kaggle_dataset/submission-Random-Forest.csv',index=False,float_format='%.4f')

## Neural Network training

In [0]:
def step_decay(epoch):
    # Learning rate scheduler object
    initial_lrate = 0.1
    drop = 0.001
    epochs_drop = 3.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate



# Model 1s
maxchannels = 1
model1s = Sequential()
timestep = 1
input_dim = 1
model1s.add(TimeDistributed(Conv1D(filters=128, kernel_size=1, activation='relu'), input_shape=(None, 1, 1)))
model1s.add(TimeDistributed(MaxPooling1D(pool_size=1)))
model1s.add(TimeDistributed(Flatten()))
model1s.add(LSTM(256, activation='relu', return_sequences=True))
model1s.add(BatchNormalization())
model1s.add(Dropout(0.2))
model1s.add(LSTM(256, activation='relu', return_sequences=True))
model1s.add(BatchNormalization())
model1s.add(Dropout(0.2))
model1s.add(LSTM(256, activation='relu'))
model1s.add(BatchNormalization())
model1s.add(Dropout(0.2))
model1s.add(Dense(maxchannels+1))
model1s.add(Activation('softmax'))

# Model 1f
maxchannels = 1
model1f = Sequential()
timestep = 1
input_dim = 1
model1f.add(TimeDistributed(Conv1D(filters=128, kernel_size=1, activation='relu'), input_shape=(None, 1, 1)))
model1f.add(TimeDistributed(MaxPooling1D(pool_size=1)))
model1f.add(TimeDistributed(Flatten()))
model1f.add(LSTM(256, activation='relu', return_sequences=True))
model1f.add(BatchNormalization())
model1f.add(Dropout(0.2))
model1f.add(LSTM(256, activation='relu', return_sequences=True))
model1f.add(BatchNormalization())
model1f.add(Dropout(0.2))
model1f.add(LSTM(256, activation='relu'))
model1f.add(BatchNormalization())
model1f.add(Dropout(0.2))
model1f.add(Dense(maxchannels+1))
model1f.add(Activation('softmax'))
model1f.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=11, average='macro')])
lrate = LearningRateScheduler(step_decay)

# Model 3
maxchannels = 3
model3 = Sequential()
timestep = 1
input_dim = 1
model3.add(TimeDistributed(Conv1D(filters=128, kernel_size=1, activation='relu'), input_shape=(None, 1, 1)))
model3.add(TimeDistributed(MaxPooling1D(pool_size=1)))
model3.add(TimeDistributed(Flatten()))
model3.add(LSTM(256, activation='relu', return_sequences=True))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(LSTM(256, activation='relu', return_sequences=True))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(LSTM(256, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(Dense(maxchannels+1))
model3.add(Activation('softmax'))
model3.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=11, average='macro')])
lrate = LearningRateScheduler(step_decay)

# Model 5
maxchannels = 5
model5 = Sequential()
timestep = 1
input_dim = 1
model5.add(TimeDistributed(Conv1D(filters=128, kernel_size=1, activation='relu'), input_shape=(None, 1, 1)))
model5.add(TimeDistributed(MaxPooling1D(pool_size=1)))
model5.add(TimeDistributed(Flatten()))
model5.add(LSTM(256, activation='relu', return_sequences=True))
model5.add(BatchNormalization())
model5.add(Dropout(0.2))
model5.add(LSTM(256, activation='relu', return_sequences=True))
model5.add(BatchNormalization())
model5.add(Dropout(0.2))
model5.add(LSTM(256, activation='relu'))
model5.add(BatchNormalization())
model5.add(Dropout(0.2))
model5.add(Dense(maxchannels+1))
model5.add(Activation('softmax'))
model5.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=11, average='macro')])
lrate = LearningRateScheduler(step_decay)

# Model 10
maxchannels = 10
model10 = Sequential()
timestep = 1
input_dim = 1
model10.add(TimeDistributed(Conv1D(filters=128, kernel_size=1, activation='relu'), input_shape=(None, 1, 1)))
model10.add(TimeDistributed(MaxPooling1D(pool_size=1)))
model10.add(TimeDistributed(Flatten()))
model10.add(LSTM(256, activation='relu', return_sequences=True))
model10.add(BatchNormalization())
model10.add(Dropout(0.2))
model10.add(LSTM(256, activation='relu', return_sequences=True))
model10.add(BatchNormalization())
model10.add(Dropout(0.2))
model10.add(LSTM(256, activation='relu'))
model10.add(BatchNormalization())
model10.add(Dropout(0.2))
model10.add(Dense(maxchannels))
model10.add(Activation('softmax'))
model10.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=11, average='macro')])
lrate = LearningRateScheduler(step_decay)

### Batchwise resampling and training

In [0]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

resample = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='not majority'), random_state=100)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X, y = resample.fit_sample(np.array(train_df['signal'][0:1000000]).reshape(-1, 1), np.array(train_df['open_channels'][0:1000000]).reshape(-1, 1))

a = pd.DataFrame(X, columns=['signal'])
a = a.to_numpy()
X = a.reshape(X.shape[0], 1, 1, 1)

b = pd.DataFrame(y, columns=['open_channels'])
b = b.to_numpy()
y = b.reshape(len(b), 1)
from numpy import array
y = to_categorical(array(y), num_classes=2)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=100)

model1s.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=2, average='macro')])
lrate = LearningRateScheduler(step_decay)
model1s.fit(x=X_train, y=y_train, initial_epoch=0, epochs=4, batch_size=256, callbacks=[lrate], verbose=1, shuffle=False, validation_data=(X_val, y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

Epoch 1/4
6745/6745 [==============================] - 100s 15ms/step - loss: 0.0180 - accuracy: 0.9939 - precision_24: 0.9939 - recall_24: 0.9939 - f1_score: 0.9939 - val_loss: 0.0130 - val_accuracy: 0.9952 - val_precision_24: 0.9952 - val_recall_24: 0.9952 - val_f1_score: 0.9952 - lr: 0.1000
Epoch 2/4
6745/6745 [==============================] - 100s 15ms/step - loss: 0.0146 - accuracy: 0.9948 - precision_24: 0.9948 - recall_24: 0.9948 - f1_score: 0.9948 - val_loss: 0.0131 - val_accuracy: 0.9952 - val_precision_24: 0.9952 - val_recall_24: 0.9952 - val_f1_score: 0.9952 - lr: 0.1000
Epoch 3/4
6745/6745 [==============================] - 101s 15ms/step - loss: 0.0144 - accuracy: 0.9951 - precision_24: 0.9951 - recall_24: 0.9951 - f1_score: 0.9951 - val_loss: 0.0127 - val_accuracy: 0.9954 - val_precision_24: 0.9954 - val_recall_24: 0.9954 - val_f1_score: 0.9954 - lr: 1.0000e-04
Epoch 4/4
6745/6745 [==============================] - 100s 15ms/step - loss: 0.0138 - accuracy: 0.9951 - preci

In [0]:
nn1f_X = np.concatenate([train_df['signal'].values[1000000: 1500000], train_df['signal'].values[3000000: 3500000]])
nn1f_y = np.concatenate([train_df['open_channels'].values[1000000: 1500000], train_df['open_channels'].values[3000000: 3500000]])

X, y = resample.fit_sample(np.array(nn1f_X).reshape(-1, 1), np.array(nn1f_y).reshape(-1, 1))

a = pd.DataFrame(X, columns=['signal'])
a = a.to_numpy()
X = a.reshape(X.shape[0], 1, 1, 1)

b = pd.DataFrame(y, columns=['open_channels'])
b = b.to_numpy()
y = b.reshape(len(b), 1)
from numpy import array
y = to_categorical(array(y), num_classes=2)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=100)

model1f.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=2, average='macro')])
lrate = LearningRateScheduler(step_decay)
model1f.fit(x=X_train, y=y_train, initial_epoch=0, epochs=4, batch_size=256, callbacks=[lrate], verbose=1, shuffle=False, validation_data=(X_val, y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

Epoch 1/4
5244/5244 [==============================] - 79s 15ms/step - loss: 0.0156 - accuracy: 0.9950 - precision_25: 0.9950 - recall_25: 0.9950 - f1_score: 0.9950 - val_loss: 0.0096 - val_accuracy: 0.9966 - val_precision_25: 0.9966 - val_recall_25: 0.9966 - val_f1_score: 0.9966 - lr: 0.1000
Epoch 2/4
5244/5244 [==============================] - 79s 15ms/step - loss: 0.0118 - accuracy: 0.9959 - precision_25: 0.9959 - recall_25: 0.9959 - f1_score: 0.9959 - val_loss: 0.0095 - val_accuracy: 0.9966 - val_precision_25: 0.9966 - val_recall_25: 0.9966 - val_f1_score: 0.9966 - lr: 0.1000
Epoch 3/4
5244/5244 [==============================] - 77s 15ms/step - loss: 0.0107 - accuracy: 0.9963 - precision_25: 0.9963 - recall_25: 0.9963 - f1_score: 0.9963 - val_loss: 0.0095 - val_accuracy: 0.9967 - val_precision_25: 0.9967 - val_recall_25: 0.9967 - val_f1_score: 0.9967 - lr: 1.0000e-04
Epoch 4/4
5244/5244 [==============================] - 78s 15ms/step - loss: 0.0107 - accuracy: 0.9963 - precision

In [0]:
nn3_X = np.concatenate([train_df['signal'].values[1500000: 2000000], train_df['signal'].values[3500000: 4000000]])
nn3_y = np.concatenate([train_df['open_channels'].values[1500000: 2000000], train_df['open_channels'].values[3500000: 4000000]])

X, y = resample.fit_sample(np.array(nn3_X).reshape(-1, 1), np.array(nn3_y).reshape(-1, 1))

a = pd.DataFrame(X, columns=['signal'])
a = a.to_numpy()
X = a.reshape(X.shape[0], 1, 1, 1)

b = pd.DataFrame(y, columns=['open_channels'])
b = b.to_numpy()
y = b.reshape(len(b), 1)
from numpy import array
y = to_categorical(array(y), num_classes=4)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=100)

model3.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=4, average='macro')])
lrate = LearningRateScheduler(step_decay)
model3.fit(x=X_train, y=y_train, initial_epoch=0, epochs=4, batch_size=256, callbacks=[lrate], verbose=1, shuffle=False, validation_data=(X_val, y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

Epoch 1/4
5731/5731 [==============================] - 86s 15ms/step - loss: 0.0384 - accuracy: 0.9892 - precision_26: 0.9894 - recall_26: 0.9890 - f1_score: 0.9893 - val_loss: 0.0266 - val_accuracy: 0.9926 - val_precision_26: 0.9927 - val_recall_26: 0.9926 - val_f1_score: 0.9927 - lr: 0.1000
Epoch 2/4
5731/5731 [==============================] - 86s 15ms/step - loss: 0.0321 - accuracy: 0.9908 - precision_26: 0.9910 - recall_26: 0.9907 - f1_score: 0.9909 - val_loss: 0.0264 - val_accuracy: 0.9926 - val_precision_26: 0.9927 - val_recall_26: 0.9925 - val_f1_score: 0.9926 - lr: 0.1000
Epoch 3/4
5731/5731 [==============================] - 85s 15ms/step - loss: 0.0331 - accuracy: 0.9904 - precision_26: 0.9906 - recall_26: 0.9902 - f1_score: 0.9905 - val_loss: 0.0260 - val_accuracy: 0.9927 - val_precision_26: 0.9928 - val_recall_26: 0.9926 - val_f1_score: 0.9927 - lr: 1.0000e-04
Epoch 4/4
5731/5731 [==============================] - 85s 15ms/step - loss: 0.0300 - accuracy: 0.9915 - precision

In [0]:
nn5_X = np.concatenate([train_df['signal'].values[2500000: 3000000], train_df['signal'].values[4000000: 4500000]])
nn5_y = np.concatenate([train_df['open_channels'].values[2500000: 3000000], train_df['open_channels'].values[4000000: 4500000]])

X, y = resample.fit_sample(np.array(nn5_X).reshape(-1, 1), np.array(nn5_y).reshape(-1, 1))

a = pd.DataFrame(X, columns=['signal'])
a = a.to_numpy()
X = a.reshape(X.shape[0], 1, 1, 1)

b = pd.DataFrame(y, columns=['open_channels'])
b = b.to_numpy()
y = b.reshape(len(b), 1)
from numpy import array
y = to_categorical(array(y), num_classes=6)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=100)

model5.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=6, average='macro')])
lrate = LearningRateScheduler(step_decay)
model5.fit(x=X_train, y=y_train, initial_epoch=0, epochs=4, batch_size=256, callbacks=[lrate], verbose=1, shuffle=False, validation_data=(X_val, y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

Epoch 1/4
7430/7430 [==============================] - 111s 15ms/step - loss: 0.0318 - accuracy: 0.9890 - precision_27: 0.9891 - recall_27: 0.9890 - f1_score: 0.9890 - val_loss: 0.0294 - val_accuracy: 0.9898 - val_precision_27: 0.9898 - val_recall_27: 0.9898 - val_f1_score: 0.9898 - lr: 0.1000
Epoch 2/4
7430/7430 [==============================] - 109s 15ms/step - loss: 0.0252 - accuracy: 0.9914 - precision_27: 0.9914 - recall_27: 0.9913 - f1_score: 0.9914 - val_loss: 0.0234 - val_accuracy: 0.9920 - val_precision_27: 0.9921 - val_recall_27: 0.9920 - val_f1_score: 0.9921 - lr: 0.1000
Epoch 3/4
7430/7430 [==============================] - 112s 15ms/step - loss: 0.0251 - accuracy: 0.9913 - precision_27: 0.9914 - recall_27: 0.9913 - f1_score: 0.9914 - val_loss: 0.0198 - val_accuracy: 0.9932 - val_precision_27: 0.9932 - val_recall_27: 0.9932 - val_f1_score: 0.9933 - lr: 1.0000e-04
Epoch 4/4
7430/7430 [==============================] - 112s 15ms/step - loss: 0.0226 - accuracy: 0.9924 - preci

In [0]:
nn10_y = np.concatenate([train_df['open_channels'].values[2000000: 2500000], train_df['open_channels'].values[4500000: 5000000]])
nn10_y = pd.Series(nn10_y)
index =np.array([nn10_y[nn10_y == 0].index[0], nn10_y[nn10_y == 0].index[1]])

nn10_X = np.concatenate([train_df['signal'].values[2000000: 2500000], train_df['signal'].values[4500000: 5000000]])
nn10_y = np.concatenate([train_df['open_channels'].values[2000000: 2500000], train_df['open_channels'].values[4500000: 5000000]])
nn10_X = np.delete(nn10_X, index)
nn10_y = np.delete(nn10_y, index)
resample = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='not majority'), random_state=100)
X, y = resample.fit_sample(np.array(nn10_X).reshape(-1, 1), np.array(nn10_y).reshape(-1, 1))

a = pd.DataFrame(X, columns=['signal'])
a = a.to_numpy()
X = a.reshape(X.shape[0], 1, 1, 1)

b = pd.DataFrame(y, columns=['open_channels'])
b = b.to_numpy()
y = b.reshape(len(b), 1)
from numpy import array
y = to_categorical(array(y), num_classes=11)

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=100)

# model10.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=11, average='macro')])
# lrate = LearningRateScheduler(step_decay)
# model10.fit(x=X_train, y=y_train, initial_epoch=0, epochs=4, batch_size=256, callbacks=[lrate], verbose=1, shuffle=False, validation_data=(X_val, y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [0]:
# y.shape
# y = to_categorical(array(y))

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=100)

# Model 10
maxchannels = 10
model10 = Sequential()
timestep = 1
input_dim = 1
model10.add(TimeDistributed(Conv1D(filters=128, kernel_size=1, activation='relu'), input_shape=(None, 1, 1)))
model10.add(TimeDistributed(MaxPooling1D(pool_size=1)))
model10.add(TimeDistributed(Flatten()))
model10.add(LSTM(256, activation='relu', return_sequences=True))
model10.add(BatchNormalization())
model10.add(Dropout(0.2))
model10.add(LSTM(256, activation='relu', return_sequences=True))
model10.add(BatchNormalization())
model10.add(Dropout(0.2))
model10.add(LSTM(256, activation='relu'))
model10.add(BatchNormalization())
model10.add(Dropout(0.2))
model10.add(Dense(maxchannels+1))
model10.add(Activation('softmax'))
model10.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False), metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=11, average='macro')])
lrate = LearningRateScheduler(step_decay)
model10.fit(x=X_train, y=y_train, initial_epoch=0, epochs=4, batch_size=256, callbacks=[lrate], verbose=1, shuffle=False, validation_data=(X_val, y_val))

Epoch 1/4
7048/7048 [==============================] - 107s 15ms/step - loss: 0.1454 - accuracy: 0.9446 - precision_28: 0.9449 - recall_28: 0.9443 - f1_score: 0.8585 - val_loss: 0.1293 - val_accuracy: 0.9459 - val_precision_28: 0.9461 - val_recall_28: 0.9457 - val_f1_score: 0.8599 - lr: 0.1000
Epoch 2/4
7048/7048 [==============================] - 106s 15ms/step - loss: 0.1199 - accuracy: 0.9536 - precision_28: 0.9538 - recall_28: 0.9535 - f1_score: 0.8670 - val_loss: 0.1083 - val_accuracy: 0.9573 - val_precision_28: 0.9574 - val_recall_28: 0.9573 - val_f1_score: 0.8701 - lr: 0.1000
Epoch 3/4
7048/7048 [==============================] - 107s 15ms/step - loss: 0.1658 - accuracy: 0.9399 - precision_28: 0.9401 - recall_28: 0.9396 - f1_score: 0.8540 - val_loss: 0.0920 - val_accuracy: 0.9648 - val_precision_28: 0.9648 - val_recall_28: 0.9647 - val_f1_score: 0.8771 - lr: 1.0000e-04
Epoch 4/4
7048/7048 [==============================] - 104s 15ms/step - loss: 0.1141 - accuracy: 0.9562 - preci

In [0]:
test_df.shape

(2000000, 3)

In [0]:
# Predicitons
sub = pd.read_csv('/content/drive/My Drive/Ion_Channel/Kaggle_dataset/sample_submission.csv')
test_df = pd.read_csv('/content/drive/My Drive/Ion_Channel/Kaggle_dataset/test_clean.csv')
test_df = np.array(test_df['signal']).reshape(test_df['signal'].shape[0], 1, 1, 1)
a = 0 # SUBSAMPLE A, Model 1s
pred_y = model1s.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 1 # SUBSAMPLE B, Model 3
pred_y = model3.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 2 # SUBSAMPLE C, Model 5
pred_y = model5.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 3 # SUBSAMPLE D, Model 1s
pred_y = model1s.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 4 # SUBSAMPLE E, Model 1f
pred_y = model1f.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 5 # SUBSAMPLE F, Model 10
pred_y = model10.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 6 # SUBSAMPLE G, Model 5
pred_y = model5.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 7 # SUBSAMPLE H, Model 10
pred_y = model10.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 8 # SUBSAMPLE I, Model 1s
pred_y = model1s.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

a = 9 # SUBSAMPLE J, Model 3
pred_y = model3.predict(test_df[100000*a:100000*(a+1)], batch_size=256)
decoded_datum = np.empty((100000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[100000*a:100000*(a+1),1] = decoded_datum.astype(int)

 # BATCHES 3 AND 4, Model 1s
pred_y = model1s.predict(test_df[1000000:2000000], batch_size=256)
decoded_datum = np.empty((1000000, 1))

def decode(datum):
    return np.argmax(datum)

for i in range(pred_y.shape[0]):
    datum = pred_y[i]
    decoded_datum[i] = decode(datum)
sub.iloc[1000000:2000000,1] = decoded_datum.astype(int)

sub.to_csv('/content/drive/My Drive/Ion_Channel/Kaggle_dataset/submission-Neural-Network.csv',index=False,float_format='%.4f')

In [0]:
ans = pd.read_csv('/content/drive/My Drive/Ion_Channel/Kaggle_dataset/submission-Neural-Network.csv')
ans['open_channels'].mean()

1.377289